In [1]:
import os
import os.path
from snowflake.snowpark import Session, version, Window
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
import openpyxl as opxl

In [2]:
snowflake_account = '<SNOWFLAKE_DEPLOYMENT>'
snowflake_user = 'USER_HOL_TIMESERIES'
snowflake_role = 'ROLE_HOL_TIMESERIES'
snowflake_warehouse = 'HOL_INGEST_WH'
snowflake_database = 'HOL_TIMESERIES'
snowflake_schema = 'STAGING'

In [3]:
passphrase=''

# Load private key using passphrase
with open('../keys/rsa_key.p8', 'rb') as key:
    p_key = serialization.load_pem_private_key(
        key.read(),
        #password=passphrase.encode(),
        password=None,
        backend=default_backend()
    )

# Set private key for connection
pkb = p_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption())

In [4]:
connection_parameters = {
    'account': snowflake_account,
    'user': snowflake_user,
    'private_key': pkb,
    'role': snowflake_role,
    'warehouse': snowflake_warehouse,
    'database': snowflake_database,
    'schema': snowflake_schema
}

# Connect to Snowflake using OAuth
session = Session.builder.configs(connection_parameters).create()

print(session.sql('select current_account(), current_warehouse(), current_database(), current_schema(), current_user(), current_role()').collect())

# Print the current version of the Snowpark library
print(version.VERSION)

[Row(CURRENT_ACCOUNT()='LL97996', CURRENT_WAREHOUSE()='HOL_INGEST_WH', CURRENT_DATABASE()='HOL_TIMESERIES', CURRENT_SCHEMA()='STAGING', CURRENT_USER()='USER_HOL_TIMESERIES', CURRENT_ROLE()='ROLE_HOL_TIMESERIES')]
(1, 13, 0)


In [5]:
df = pd.read_excel(open('../data/production_data/dataset/Volve production data.xlsx', 'rb'),
              sheet_name='Daily Production Data')

In [6]:
df_snow = session.create_dataframe(df)
# Write out data frame to table
df_snow.write.mode("overwrite").save_as_table("RAW_TS_PRODUCTION_DATA")
df_snow.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"DATEPRD"            |"WELL_BORE_CODE"  |"NPD_WELL_BORE_CODE"  |"NPD_WELL_BORE_NAME"  |"NPD_FIELD_CODE"  |"NPD_FIELD_NAME"  |"NPD_FACILITY_CODE"  |"NPD_FACILITY_NAME"  |"ON_STREAM_HRS"  |"AVG_DOWNHOLE_PRESSURE"  |"AVG_DOWNHOLE_TEMPERATURE"  |"AVG_DP_TUBING"  |"AVG_ANNULUS_PRESS"  |"AVG_CHOKE_SIZE_P"  |"AVG_CHOKE_UOM"  |"AVG_WHP_P"  |"AVG_WHT_P"  |"DP_CHOKE_SIZE"  |"BORE_OIL_VOL"  |"BORE_GAS_VOL"  |"BORE_WAT_VOL"  |"BORE_WI_VOL"  |"FLOW_KIND"  |"WELL_TYPE"  |
------------------------------------------------------------------------